In [2]:
# default_exp tasks

In [27]:
#hide
from nbdev.showdoc import *
from datetime import datetime, date, time, timedelta

In [2]:
#export 
from collections import namedtuple
properties = ['task',
              'Estd_Time',
              'Strt_Date',
              'Time_Takn',
              'Comp_Date',
              'Satsfactn'
             ]
defaults = ['?']*len(properties-1)
Task = namedtuple('Task', properties, defaults=defaults)

- task: str of the task
- [ ] Estd_Time: str of the form '12m' which then gets converted into timedelta object
- [ ] Start_Date: today's date or scheduled date of the form 'dd/mm/yy' or 'tomorrow'
- [ ] Time_takn: str of the form '12m'

In [3]:
#export
t = Task(45,10);

In [4]:
#export 
def task_scheduler(x: 'list of strings of tasks'):
    

ask(x='?', y='?', z='?')

In [28]:
tasks = Task('asuha')

In [7]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted Tasks.ipynb.
Converted index.ipynb.
